# Model Training

## Table of Contents
- Evaluation Metrics

## Evaluation Metrics

Below are the key classification metrics that we will use to evaluate our model performance:

#### Accuracy
Accuracy provides a ratio of correctly predicted observations to the total observations. 
**Formula**:
$$\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}$$

#### Confusion Matrix and Related Terms
The confusion matrix is a table layout that allows visualization of the performance of the algorithm, where each number in the matrix represents:
- **TP (True Positives)**: Correctly predicted positive observations.
- **TN (True Negatives)**: Correctly predicted negative observations.
- **FP (False Positives)**: Incorrectly predicted as positive.
- **FN (False Negatives)**: Incorrectly predicted as negative.

#### Precision, Recall, and F1-Score
- **Precision**:
$$ \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} $$
Precision measures the accuracy of positive predictions.

- **Recall** (or Sensitivity or TPR):
$$ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} $$
Recall measures the ability of a model to find all the relevant cases (all positive samples).

- **F1-Score**:
$$ \text{F1-Score} = 2 \times \left( \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} \right) $$
The F1-Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. A high F1-score shows a model can classify the positive class correctly, while not misclassifying many negative classes as positive.

<br>

## Import data & train_test_split

In [ ]:
import pandas as pd
df = pd.read_csv("data/cleaned_df_v2.csv", index_col=False)
df = df.iloc[: , 1:]
df.head()

,year_month,us_state,make,phone_series,phone_launch_date,model_family,predecessor,successor,model_age_days,model_age_months,...,closing_subs_monthly,filed_claims,claims,claims_swap,claims_replacement,ir_rate_swap,ir_rate_replacement,ir_rate_monthly,churn_rate,churn
0,2019-01-01,California,Oppo,OPPO A10,2017-11-01,OPPO A12,First Model,OPPO A15,456,14,...,137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2019-02-01,California,Oppo,OPPO A10,2017-11-01,OPPO A12,First Model,OPPO A15,484,15,...,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2019-03-01,California,Oppo,OPPO A10,2017-11-01,OPPO A12,First Model,OPPO A15,515,16,...,162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2019-04-01,California,Oppo,OPPO A10,2017-11-01,OPPO A12,First Model,OPPO A15,545,17,...,173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2019-05-01,California,Oppo,OPPO A10,2017-11-01,OPPO A12,First Model,OPPO A15,576,18,...,179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
from sklearn.model_selection import train_test_split

X = df[df.columns.drop('closing_subs_monthly')]
y = df['closing_subs_monthly']

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=3244
    )

## Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


ValueError: could not convert string to float: '2023-06-01'